In [17]:
import gensim
wvmodel = gensim.models.KeyedVectors.load_word2vec_format(r'GoogleNews-vectors-negative300.bin',binary=True)

In [87]:
#import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
import os
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import RegexpTokenizer
from os import listdir
from os.path import isfile, join
import numpy as np

vocab = (open("mini_newsgroups/vocabulary.txt","r", encoding="utf8", errors='ignore')).read()
def process_doc(doc):
    doc = doc.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(doc)

    stop_words = set(stopwords.words('english')) 
    filtered_sentence = []
    for w in word_tokens:
        if (not w in stop_words) and ("\n"+w+"\n" in vocab) and (w in wvmodel):
            filtered_sentence.append(w)
            
    return filtered_sentence

In [88]:
def parser(d, f):
    file = open('./mini_newsgroups/'+d+'/'+f, 'r', encoding="utf8", errors='ignore')
    string = file.read()
    file.close()
    return process_doc(string)

In [89]:
dirs = [d for d in os.listdir('mini_newsgroups') if not isfile(join('mini_newsgroups', d))]
train_X = []
train_Y = []
namefile = []
for d in dirs:
    for root, dirs, files in os.walk('./mini_newsgroups/'+d):  
        index_to_pick = [i for i in range(len(files))]
        np.random.shuffle(index_to_pick)
        for i in index_to_pick[0:100]:
            train_X.append(parser(d, files[i]))
            train_Y.append(d)
            namefile.append(files[i])

In [85]:
from collections import Counter

for i in range(len(train_X)):
    if len(set(train_X[i])) >= 150:
        words = Counter(train_X[i]).most_common()
        train_X[i] = []
        picked = 0
        for h in range(150):
            if(words[h][1] > 1):
                for j in range(words[h][1]):
                    train_X[i].append(words[h][0])
                picked+=1
            else:
                break
        indexes = [x for x in range(picked,len(words))]
        np.random.shuffle(indexes)
        for h in range(150-picked):
            train_X[i].append(words[indexes[h]][0])

In [90]:
import pickle 
obj = ['newsgroup_dataset', train_X, train_Y] 
f = open('newsgroup_dataset.pckl', 'wb') 
pickle.dump(obj, f) 
f.close()

In [84]:
from gensim.corpora.dictionary import Dictionary
def wmdistance(document1, document2, wvmodel):
        from pyemd import emd

        # Remove out-of-vocabulary words.
        len_pre_oov1 = len(document1)
        len_pre_oov2 = len(document2)
        remember_me = document2
        document2 = ['__qxvca^&3fd?#_!$' +  str(i) for i in range(len(document2))]
        diff1 = len_pre_oov1 - len(document1)
        diff2 = len_pre_oov2 - len(document2)
        if diff1 > 0 or diff2 > 0:
            logger.info('Removed %d and %d OOV words from document 1 and 2 (respectively).', diff1, diff2)

        if not document1 or not document2:
            logger.info(
                "At least one of the documents had no words that were in the vocabulary. "
                "Aborting (returning inf)."
            )
            return float('inf')

        dictionary = Dictionary(documents=[document1, document2])
        vocab_len = len(dictionary)

        if vocab_len == 1:
            # Both documents are composed by a single unique token
            return 0.0

        # Sets for faster look-up.
        docset1 = set(document1)
        docset2 = set(document2)

        wordvecs_random = {document2[i]: remember_me[i] for i in range(len(remember_me))}

        # Compute distance matrix.
        distance_matrix = np.zeros((vocab_len, vocab_len))
        for i, t1 in dictionary.items():
            if t1 not in docset1:
                continue

            for j, t2 in dictionary.items():
                if t2 not in docset2 or distance_matrix[i, j] != 0.0:
                    continue

                # Compute Euclidean distance between word vectors.
                distance_matrix[i, j] = distance_matrix[j, i] = np.sqrt(sum((wvmodel[t1] - wordvecs_random[t2])**2))

        def nbow(document):
            d = np.zeros(vocab_len)
            nbow = dictionary.doc2bow(document)  # Word frequencies.
            doc_len = len(document)
            for idx, freq in nbow:
                d[idx] = freq / float(doc_len)  # Normalized word frequencies.
            return d

        # Compute nBOW representation of documents.
        d1 = nbow(document1)
        d2 = nbow(document2)

        # Compute WMD.
        return emd(d1, d2, distance_matrix)
